In [1]:
import pickle
temp = pickle.load(open('cust_data.pickle', "rb"))

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
# df = pd.DataFrame(columns=["cust_id", "sources", "train_mask", "test_mask", "cust_data"])
rows = []
for user in temp:
    new_row = {'cust_id':[user],'cust_data':temp[user]['cust_data']}
#     new_row = {'cust_data' :temp[user]['cust_data']}
    rows.append(new_row)

# print(df)

In [3]:
print(temp['82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1'])

{'cust_data': [{'cust_id': '82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1', 'debit_credit': 1, 'tx_date': 0, 'tx_time': 13, 'tx_type': 1, 'tx_amt': 6108.0, 'exchg_rate': 28.555, 'info_asset_code': 20, 'fiscTxId': 6, 'txbranch': 349, 'cross_bank': 0, 'ATM': 1, 'data_source': 'dp'}, {'cust_id': '82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1', 'debit_credit': 0, 'tx_date': 0, 'tx_time': 18, 'tx_type': 1, 'tx_amt': 87041.0, 'exchg_rate': 4.337, 'info_asset_code': 20, 'fiscTxId': 6, 'txbranch': 349, 'cross_bank': 0, 'ATM': 1, 'data_source': 'dp'}, {'cust_id': '82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1', 'debit_credit': 0, 'tx_date': 0, 'tx_time': 14, 'tx_type': 1, 'tx_amt': 90172.0, 'exchg_rate': 28.495, 'info_asset_code': 20, 'fiscTxId': 6, 'txbranch': 349, 'cross_bank': 0, 'ATM': 1, 'data_source': 'dp'}, {'cust_id': '82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1', 'debit_credit': 0, 'tx_date': 0, 'tx_time': 19

In [4]:
print(rows[0]['cust_data'])

[{'cust_id': '82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1', 'debit_credit': 1, 'tx_date': 0, 'tx_time': 13, 'tx_type': 1, 'tx_amt': 6108.0, 'exchg_rate': 28.555, 'info_asset_code': 20, 'fiscTxId': 6, 'txbranch': 349, 'cross_bank': 0, 'ATM': 1, 'data_source': 'dp'}, {'cust_id': '82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1', 'debit_credit': 0, 'tx_date': 0, 'tx_time': 18, 'tx_type': 1, 'tx_amt': 87041.0, 'exchg_rate': 4.337, 'info_asset_code': 20, 'fiscTxId': 6, 'txbranch': 349, 'cross_bank': 0, 'ATM': 1, 'data_source': 'dp'}, {'cust_id': '82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1', 'debit_credit': 0, 'tx_date': 0, 'tx_time': 14, 'tx_type': 1, 'tx_amt': 90172.0, 'exchg_rate': 28.495, 'info_asset_code': 20, 'fiscTxId': 6, 'txbranch': 349, 'cross_bank': 0, 'ATM': 1, 'data_source': 'dp'}, {'cust_id': '82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1', 'debit_credit': 0, 'tx_date': 0, 'tx_time': 19, 'tx_type': 1

In [5]:
from sklearn.feature_extraction import DictVectorizer
small_row = rows
vector = DictVectorizer(sparse=False)
for x in small_row:
#     print(x)
#     vector.fit(x['cust_id'])
    vector.fit(x['cust_data'])
# X = vector.fit_transform(small_row)

In [6]:
# result = vector.transform(small_row)
result = np.empty
done = False
for y in small_row:
    if(done == False):
#         print("here")
        result = vector.transform(y['cust_data'])
        done = True
    else:
#         print("done")
        result = np.concatenate((result,vector.transform(y['cust_data']) ), axis=0)
#     np.concatenate((a, b), axis=0)
#     print(type(y['cust_data']))
    
#     print(type(y['cust_id']))
#     print(type(y['cust_data']))
#     y['cust_id'] = vector.transform(y['cust_id'])

In [7]:
df = pd.DataFrame.from_records(small_row)
print(type(df.iloc[0]))

<class 'pandas.core.series.Series'>


In [8]:
result = tf.expand_dims(result, axis=-1)

In [9]:
# model building
from tensorflow.keras.models import Model
from tensorflow.keras.constraints import unit_norm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, MaxPooling2D,AveragePooling1D, GlobalMaxPooling2D, Conv2D
from tensorflow.keras.layers import ReLU, Softmax
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization,Flatten, Activation, GlobalMaxPooling1D, Conv1D, LayerNormalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax
from tensorflow.keras import regularizers

model = Sequential()
model.add(Conv1D(filters=32,kernel_size=(1),strides=(1),padding='same', kernel_initializer='he_normal', input_shape=(None, 35,1), activation='relu'))
model.add(LayerNormalization())
model.add(Conv2D(filters=32, kernel_size=(1,1),strides=(1,1), padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.001)))
model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1), padding = 'same'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# model.add(Conv2D(filters=64, kernel_size=(1,1),strides=(1,1),  padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.001)))
# model.add(LayerNormalization())
# model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1), padding = 'same'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

# model.add(Conv2D(filters=64, kernel_size=(1,1),strides=(1,1), padding='same', activation='relu',kernel_regularizer=regularizers.l2(l=0.001)))
# model.add(MaxPooling2D(pool_size=(1,1), strides=1, padding = 'same'))
# model.add(BatchNormalization())

# model.add(Conv2D(filters=64, kernel_size=(1,1),strides=(1,1),  padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1), padding = 'same'))
# model.add(BatchNormalization())

# model.add(Conv2D(filters=64, kernel_size=(1,1),strides=(1,1),  padding='same', activation='relu'))
# model.add(AveragePooling2D(pool_size=(1,1), padding = 'same'))
# model.add(BatchNormalization())
# model.add(AveragePooling2D(pool_size=(1,1), padding = 'same'))
# model.add(GlobalMaxPooling2D())

# model.add(Flatten()) 
# model.add(Dense(35, activation='relu', kernel_initializer='normal'))
model.add(Dropout(0.4))

# model.add(Dense(35, activation='relu', kernel_initializer='normal'))
# model.add(Dropout(0.4))

# model.add(Dense(35,activation='softmax'))
# model.add(Dense(units=35, activation='linear',input_dim=35))
# model.add(Dense(35))
# model.add(Dropout(0.1))
# model.add(Dense(35))
# model.add(Dropout(0.1))
# model.add(Dense(35,activation='softmax'))

print(result.shape)
# model.build(result,result)
model.summary()

lr_schedule = ExponentialDecay(initial_learning_rate=1e-2,decay_steps=100000,decay_rate=0.9)
optimizer = Adamax(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

(3114825, 35, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, None, 35, 32)      64        
_________________________________________________________________
layer_normalization (LayerNo (None, None, 35, 32)      64        
_________________________________________________________________
conv2d (Conv2D)              (None, None, 35, 32)      1056      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, None, 35, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, None, 35, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, 35, 32)      128       
_________________________________________________________________
dropout_1 (Dropout)          (None, Non

In [10]:
from tensorflow.keras.callbacks import CSVLogger
# csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 1
batch_size = 256

# training!
history = model.fit(result,result,
                    epochs=epochs, 
                    batch_size=batch_size, 
#                     callbacks=[csv_logger],
                    use_multiprocessing=True,
#                     validation_data = (fakeX_test, fakey_test)
                       )
print('training finish')

12168/12168 [==============================] - 403s 33ms/step - loss: nan - accuracy: 0.7600
training finish


In [12]:
sample = pd.read_csv('Dataset/sample_submission.csv')
custinfo = pd.read_csv('Dataset/public_train_x_custinfo_full_hashed.csv')
all_keys = sample.alert_key
print(all_keys)
test_cust_ids = custinfo[custinfo['alert_key'].isin(all_keys)].cust_id
print(test_cust_ids)

0       357307
1       376329
2       373644
3       357668
4       354443
         ...  
3845    364485
3846    363155
3847    368710
3848    358067
3849    372119
Name: alert_key, Length: 3850, dtype: int64
0       82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a...
1       b212d14cb35676926682b2cf849e295d948888f556c07e...
2       e5b0002791c7852644a2730abeaa893cdf14a072ef7812...
3       74214c478dc6519fbefe4bc31693865bdcd698ab974b64...
4       0340e7611f0d82c3cb87e6194fa14bb2ccf8afbf1b3418...
                              ...                        
1840    18ee644a371548e9780d701aaa7e0c8c42a7794cdee755...
1841    7f69fa9eab8f397d367e2bb61ee1fa008999a0aab91e06...
1842    12c9e6d35500d2a96fc2b22a9da8e3deb6048de515a16e...
1843    d24d46c19002ab1f9a02801af5e4be6a154b3c5adc0417...
1844    7224e3a067b5e90c9b2a5bdbcfe81884505ab44f0566ae...
Name: cust_id, Length: 1845, dtype: object


In [13]:
alert_to_id_dict = {}
for x in all_keys:
    if(custinfo.loc[custinfo['alert_key'] == x].empty == False):
        cust_id = custinfo.loc[custinfo['alert_key'] == x]['cust_id']
        print(cust_id.values[0])
        alert_to_id_dict[cust_id.values[0]] = x
# print(alert_to_id_dict)

1d69b4daf9c5c8b8d68b1776193b6e80bf2e84c999d0387d2866830d4848bf75
8b51184740375f7ccdd68484aeeaca44c5892818eda908eed29459de7ccbdf76
c6def618ad861703c025be4f41bdf7569310228ae93aeffc73088c520c79558d
ca010e402907caab7f043733db0de6c8c29c58d73a0848680771c5ce12b26fb4
52ae0057ef14fb63f4428ffd8a57e8e14b598dad0f638420d08ee1c2c3e58b8d
987fa9033f6d7d6750dac714e178516755e75bd87dcedaee5dea1d08386a7104
cb1acc785bf01497b8a08d43da3bda0220f60188991af29932973d0ed03dba2e
823fc5ce48cd827628ce0d1c574e6c3582cb772cf6173a9384df22d56bb72f0c
49f3128a76c44b29d9de1e3e41185cb2f091c3762501e080795c2bb2f8c49622
d0afa52e9e57910af5bff82151343fd65bf522ed1a164118ac4785fc3b492137
04ac4e9d22843711c4c1276de3b55c377016e2158f280f1281ca3b17b018e994
a3aac9909fe00ed8c8fcd67573d93baa6e1c8cbf2cbc7f55e648bf0037e2d5a6
d11b9cd483cf1fe9d2c26de44af17d86707e1b9159c2b2a5235ef01aa8aa9376
112694ce0f5742d4f3831ecc99f877624ca17c65f04d2c137d475a9ee5b936ac
c9e94a43acd8c0780df93b1777859da7a6e31c692ef0737848dae56c0db32c2c
08d2b4ae3c03515cac79354b8

b8cb473b68019c3fa08007da3c927b6381b96d730fd3e8510daebfb678d7308b
24110a2ec5a36adb9dbb6540faf32719b82f3b181cc37ceab0041e2a35a2cc87
fb832de21bafdfae18e043fa51952a719900b74b51924d48d5079fb64c690882
33ff49ea8a07c6d1b7cc203dcc3638ebde62dfb960a1697c8e5a55dffefc1cb6
06c5ea5ccda4dfd8839c0dec8646fb3071d2c02883ef8a9f715aaa74f38d2419
95762212daf16ed6db11583d20dbf27f231a7e6fda9f498e24a5ae9398834430
e507c2e670cebd5699fd0e115d3fb5b4d4767992474fd7d2a14d1642258dee53
d965728f86144f30286f2eb7bcfe51d5cb2bc9c228accbdbc858136d710676a3
304d1c4943cb83843d54cf044f1a919545e89f6e6bf8acd060903292e5c988db
8a89be5b38f76afa6deeb419c9efd97fca86463faba234f3d798db28be3a91f6
fff322436b556032e07690f12456d9eff3588c269d6210405947733f1a9413fd
04ac4e9d22843711c4c1276de3b55c377016e2158f280f1281ca3b17b018e994
7a0f52225c08808e07753ca0a975a08a22dce5dfbd6445223eed8c3a70ed84c3
8af18d87937633313c1040db8b01bd51bced5a9c041bff85f262aa64dee63864
d24d46c19002ab1f9a02801af5e4be6a154b3c5adc04172e372c16ca446d736b
a0a33aadc2db812079e389806

54120a833a4e411f5e15fd3fbdf32e9cd66e000cd6f5ba24bc910ce5e3d46275
b1376e9558831e2bdd7456be697ffe67e7186a15c6c280db023cd790f1b975e9
196d8720ded07e7f8f7f457ce45cb712bcecb86084bebcc1696d4356c1451b63
cd1bfab2b5aede00574f4308821d4ea38f32460f3cf489ff4375eefc765bbb87
49b722b2fe31df90a8668e3f894d3d5704e8141cbb0d5d1346c168166a0b4b63
098cdef7ce63ba45aa14b481f041d9fce5f99b9fdf2da0d625a61b752cfdaae3
1a6f12af41d4127dcdbce007eb4c2b5689fd3e3dcba7c0627dfdeeb049804be3
6892e5959907cc8209491076034f1e021a618123ecd3935bf7122ff5804f9214
5c8203758d604e065e40f5192d41077acdb92b1121bd5358436411be10072351
d94a32cd6746a73e9d2401264b58a067205754179d82faa9bba1a408be9588b5
f284d9a1f047636cf371af0d1e2cc1a8583c413aca0e347538758b914e00eeec
308652839f3127d4323f5983349c61c150c75416a3fe14dbb3cb71dec8ef615b
9b46e90b9bf951b14c68702c5a3fff2fe28fbc061fc6e5e881c95ba35410e673
ef69202db6a2dec5d06ae75f12731ca127d9005f279106596101dc6c0a32d5c6
40fa92acb89d1ebb319f0f200d31a5fcbdfcbd8ad432d732a880545a69f2c5e8
54c04f1809054b1558b09dea4

e87f2b72e3299c9230f62e45f6342c343886b1711c9f1b053ad4307d86ba25a4
c7623a65c98c6f08aeebc88fcafe1ebc7c47e36e3fe4e1c54cbf685ff89a69c3
79caee7bb4182d7ff1667ca33fa160e0d3592f1ab8076bb07a20e8f379134205
76e189b30d7798738fc335d50caf71f4944a65df06e4dc6c7c869af272fb127a
f5f3b14c8676b2766e27c35533ee866a229341ce7fdd3017a5886d1a621885b3
4dfeca8a959845f1ed0a40340b8c53525bb477c5c37a7068d6383515d211c881
25afe78936124d84ec09783a1afdff49da24ab171f1b5d248380225b6fab1412
fc209a9b988e9cbdbd06cb8cd302c340919bb441fe343a66ac494d84b8038852
24110a2ec5a36adb9dbb6540faf32719b82f3b181cc37ceab0041e2a35a2cc87
3513b38b20fd02573807e4a414ed7341e3bd0844fc525d5294adae63e5c61af2
b212d14cb35676926682b2cf849e295d948888f556c07e854c4cdad9f32f5084
01c705b457297805c1c08137cb3adb164b8f393c265db2c3aff0a5484a703302
36a4eb2973ced2ee87441a53dd7989d31e3da2af95360197a6fac9bd6f9f5389
8bef583a6cc51c7e1f06257e6ca45c223d1c2994efb51ac1538921af33dfb79a
4b47598def63e8c71fe0012e487edd3d88e6c02ab5b76703cdd89ce91786e0cc
a0a33aadc2db812079e389806

76e189b30d7798738fc335d50caf71f4944a65df06e4dc6c7c869af272fb127a
6d25805b1b08a8638109af3af9ec201241909826b9f0ee736cdeacd3fb12f80e
67657692d6856504a9ab96bae3bbb21effe7f59f8aa0a4f34b305c032b68e54e
a1b1bba7f1f3d8f251f43b0ba9991a8ffdc3ec3036bb56a8301ba0748120b14c
cbae27ede30fb485f5632fea768f4b999df95cdad39c91f359155d8ec6e099b3
24b47856a2e1ba7862c9366a4662738d428d1e2d456a9f1417972aace87331c3
e6ddaec886626e75908a9b657bc4a0ca704ec28a9f88b9e6fb2103d5bab6876c
74214c478dc6519fbefe4bc31693865bdcd698ab974b6435bb8fa842a3ed4fd8
659c6503ca6a1e50e92885ba94ca0efc5cc3881fccc8041d5b0f82a09d1c6081
81ecc7ab1a28a706816d641333a50f496f820a9011b136e9736e2c2f2a7781fc
95a6c0fd787a7f03473b58897e0d696d4ba27c61003f9277493d438798c7191a
fa1919bc7f2d48288a1eab1b97b2a495557a27789cbea7484fa75596ed18e3a5
2a0fdc582840f1ef3ad9b597d05f6d73533a063ef818966ef0b75bc84f3465bb
33ff49ea8a07c6d1b7cc203dcc3638ebde62dfb960a1697c8e5a55dffefc1cb6
161b6ca5a8193e5bdaff50bcfd82993728ebb26d5d45625f1af7d271482ca4b5
03bbccf0289660b1a481fcfa6

54120a833a4e411f5e15fd3fbdf32e9cd66e000cd6f5ba24bc910ce5e3d46275
a6d01e506564ae99d2ddaa260545d9dfcd124004af63d97781a608b5c0119d61
7d163b5438256a353fc6328906d8a59fe1f7e072755cf783fac6ecdc428dc83a
24af5887ebccd3c048b954fb3fbbb6136171f2719e9cf5b175d34edc2e0f53f1
7224e3a067b5e90c9b2a5bdbcfe81884505ab44f0566aecd3940244d1b99d817
8af18d87937633313c1040db8b01bd51bced5a9c041bff85f262aa64dee63864
11d0a68dfba18e8857e71d162fa0ee5f1960ae1056fc3d735754b9d8ca0d56f4
10415f999cd889a1353fd418371681b04b63b9ba0a0de58198ef836d2b4f7d7c
79caee7bb4182d7ff1667ca33fa160e0d3592f1ab8076bb07a20e8f379134205
8ad6c338a1078ea5139423c7243914c0201f4afa21def82600d46594bff8e852
24af5887ebccd3c048b954fb3fbbb6136171f2719e9cf5b175d34edc2e0f53f1
279922d10aeb65f598b19a339619f9dfa126e39a0a93f35901886f0ecd325b52
ff15b20b2fb46fe8d371ee7256633ed95c1c5f6c50760694bf3f32e1385157bd
8a8a3d19fbce6501b156018ffcf902470d07070bc55f69a3bd0c4c76be6b670a
40fa92acb89d1ebb319f0f200d31a5fcbdfcbd8ad432d732a880545a69f2c5e8
515aa3c855e28116e4f2e2715

cc61734a64024edf0437d6ae76f54458cdcb0b5da9a85a20f81e0290aaf5db3b
27b8e8e3bec08da1248b3397834ffb4a9932bfb2b3b8795c6f6e5b9bc61aad41
823fc5ce48cd827628ce0d1c574e6c3582cb772cf6173a9384df22d56bb72f0c
67f5cb11e82fbee902dbb29f52e5b40e87acc4f9d28916b6a89603c050ecedbe
8c7a8bb21706ac20aa054090d606112d6c7b9d0320026b4cdcd9415adb906c41
462ac2db5db074267cc8bd9e1861c95e57e3f30cb877aa6184eb7cf3798366e6
9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33fa1f70867748596bd5
d44bf5a06abb0782f551aa67a935734f88e3b419cb61decdf6882df4b30139f0
1ce1c66d64dd1ca22bf0770832213f81ba0a531ba63f3f2195233a3a26f79887
823fc5ce48cd827628ce0d1c574e6c3582cb772cf6173a9384df22d56bb72f0c
a9737197cb45f04f4dec6098f5185ad95006c492d66da0e7e2434ed7e2ed4452
e29643f9e971db8c2a2af05bf831b465b2d2dddf2f035d34ce863bf36083b3eb
7b1ea66e757368e4b4371100aad46609c47499811b62a80507ba4006f0bbab32
6fee9aadb7a56cffe45f461539a3a837f7c9aa281c1d14eb27ace6ec15e5604e
cd6a47e39eacd9f07f51fd68794bf19d016bec56dcae168788eec1d664a545d8
85560737ffda7254ba16b71e4

a0a33aadc2db812079e38980653e7d6606fdb38ff240fede5af640721e5609eb
b212d14cb35676926682b2cf849e295d948888f556c07e854c4cdad9f32f5084
ee136f2ada20196bf017d3628f2850384322a7af5eda0ff1c4fd4b41c2021a70
7042969fb54f2f114792ee68d824ad0500464c12f4e5784eec6d696c3af7a05c
7ea8d681ee9394e27e6064d23d0918ae9b67d1b21dbae9744ed66ef8c8a526fb
7224e3a067b5e90c9b2a5bdbcfe81884505ab44f0566aecd3940244d1b99d817
54120a833a4e411f5e15fd3fbdf32e9cd66e000cd6f5ba24bc910ce5e3d46275
304d1c4943cb83843d54cf044f1a919545e89f6e6bf8acd060903292e5c988db
01c705b457297805c1c08137cb3adb164b8f393c265db2c3aff0a5484a703302
f22e925ea421ac8ef44b16b7e382486f8971be529b6fbae9facf567f6f831c19
8e948677515036b40199bf7a4f8d9efa3a6bed3da4f28241f7f2651d52085d6f
de86b8682ff4c4b144be64b61f5e117d88317128b3ce6789605dcac7d9856398
4ee62fe681eadacf42a02da457668f042b2f4560a8ce2de5bddf2ad939222480
24110a2ec5a36adb9dbb6540faf32719b82f3b181cc37ceab0041e2a35a2cc87
3182ab9ecb0e19b96a0a68723490c01cc46dbc1e8d8a31801c9a010b436eee89
3182ab9ecb0e19b96a0a68723

16fbc659bf1f2d26998be3a44b843c842cb9a753a229b4dd671bc55ddae89ffd
a3f3ab8ef1dc14645641197c93c50af693039cc6f2b879ae5c37b2d3b8dbd8dc
08d2b4ae3c03515cac79354b8f30e3287a8d427f8af2d6ece78074fa45c148d0
158f652568e23e7e4a8e1c7cbf8c8987d53f23049a1e076f0dc30a993e979382
3b2d46248f7275197706cec8b10b82ffdcf48b71aa3ae09f070a8fd5c3511d56
16696b34381a5249ef7ffd7dd498ae10e575ad9790601c53940ad6a2020487ce
c90197eb61812d8b41228fa0d602f9d5062b10f4c18feca4e32d28dd6e16d4b5
eddfc6bc7e209e2a40808c3fb07e08b3ee0a199e0a8552e3dc4952b414332d4b
c1fd75c8241202e9fbe747d44851dd5ba082b5e368b1b3b5f62a13f96e460b27
fe0b3b09a3b8c5435d55a286ecd4fb416e96d3231abdf27c975bf8606a0b559c
ad401c32c48cf7a2d070751ca67ed2678f07cc9cec801b6be85b6358d038afdf
cd1bfab2b5aede00574f4308821d4ea38f32460f3cf489ff4375eefc765bbb87
b8cc477cdb4db116c5f53248b99df135b8b018646c7d81f6a726225c98d93768
f41c0860cf0cb5e7b72b0ced16ab427a06fc0ad3fc0f710d1f2e1695bd4571bf
823fc5ce48cd827628ce0d1c574e6c3582cb772cf6173a9384df22d56bb72f0c
33ff49ea8a07c6d1b7cc203dc

In [14]:
print(test_cust_ids[0])
print(alert_to_id_dict[test_cust_ids[0]])

82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1
360226


In [15]:
print(test_cust_ids[0])
alert_key_rows = []
for ids in test_cust_ids:
    alert = alert_to_id_dict[ids]
    new_row = {'alert_key':alert,'cust_data':temp[ids]['cust_data']}
    alert_key_rows.append(new_row)
# print(alert_key_rows[0])

82595ac69158ae08d34156784bdec0d9e2ca5b242b6d2a8542c040b98feb8bd1


In [16]:
# test_rows = []
# for x in rows:
# #     print(x['cust_id'][0])
#     for y in test_cust_ids.items():
# #         print(y[1])
#         if (y[1] == x['cust_id'][0]):
# #             print("true")
#             test_rows.append(x)
# # print(test_rows[0])

In [17]:
# test_result = np.empty
# done = False
# for y in test_rows:
#     if(done == False):
# #         print("here")
#         test_result = vector.transform(y['cust_data'])
#         done = True
#     else:
# #         print("done")
#         test_result = np.concatenate((test_result,vector.transform(y['cust_data']) ), axis=0)

In [18]:
# final = model.predict(test_result, batch_size=128)
# print(final.length())

In [19]:
for x in alert_key_rows:
    x['cust_data'] = vector.transform(x['cust_data'])
for x in alert_key_rows:
    x['cust_data'] = model.predict(tf.expand_dims(x['cust_data'], axis=-1), batch_size=128)

In [20]:
print(alert_key_rows[0]['cust_data'])

[[[[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  ...

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]]


 [[[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  ...

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]

  [[-2.215e-42 -2.215e-42 -2.215e-42 ... -2.215e-42 -2.215e-42
    -2.215e-42]]]


 [[[-2.215

In [21]:
# flattened = temp_alert['cust_data'].flatten()
from scipy.stats import norm
def get_probability(cust_data):
    normalized = np.random.normal(cust_data)
    flattened_normalized = normalized.flatten()
    mean = np.mean(flattened_normalized)
    std = np.std(flattened_normalized)
    maxval = max(flattened_normalized)
    probability_pdf = norm.pdf(maxval, loc=mean, scale=std)
    return round(probability_pdf, 7)
# print(probability_pdf)

In [22]:
def get_probability2(cust_data):
    normalized = np.random.normal(cust_data)
#     print("normalized")
#     print(normalized)
    flattened_normalized = normalized.flatten()
    flattened_normalized = np.nan_to_num(flattened_normalized, nan=0)
#     print("flattened")
#     print(flattened_normalized)
    mean = np.mean(flattened_normalized)
#     print("mean")
#     print(mean)
    std = np.std(flattened_normalized)
#     print("std")
#     print(std)
    maxval = max(flattened_normalized)
#     print("maxval")
#     print(maxval)
    probability_pdf = norm.pdf(maxval, loc=mean, scale=std)
    return round(probability_pdf, 7)

In [23]:
print(get_probability2(alert_key_rows[0]['cust_data']))

1.1e-06


In [24]:
final_ans = {}
for x in alert_key_rows:
    final_ans[x['alert_key']] = get_probability2(x['cust_data'])

In [25]:
print(final_ans)

{360226: 9e-07, 360230: 1.27e-05, 352254: 0.0001542, 359626: 1.2e-06, 352282: 0.0001669, 352291: 8.1e-06, 357275: 6e-07, 352301: 0.0006651, 353043: 2.2e-06, 359253: 1.3e-06, 354622: 7e-07, 352309: 2.54e-05, 352312: 0.0001568, 355395: 1.4e-06, 352320: 2.1e-06, 355413: 4.4e-06, 352330: 9.32e-05, 352331: 0.0001254, 360287: 1.3e-06, 358624: 1.28e-05, 359673: 8e-06, 353083: 7.1e-06, 358067: 8.1e-06, 359827: 2.4e-06, 352470: 3.7e-06, 352472: 4.6e-06, 352476: 9.75e-05, 358213: 8.2e-06, 352484: 0.0002548, 353394: 1.7e-06, 363583: 6.3e-06, 354044: 6.4e-06, 356777: 1e-07, 352494: 1.5e-06, 352495: 3e-07, 364036: 1.2e-06, 355563: 9.1e-06, 359858: 4.48e-05, 352507: 1.02e-05, 352508: 9e-06, 352520: 9.5e-06, 352522: 1e-06, 352557: 5.68e-05, 352558: 0.0001629, 363121: 6.4e-06, 355671: 2e-07, 352658: 7e-06, 352660: 3.5e-06, 352670: 0.0001196, 352680: 0.0005084, 358451: 1.9e-06, 354430: 1e-07, 355794: 7.2e-06, 354432: 2.25e-05, 364309: 1e-07, 352692: 6.16e-05, 361290: 6.7e-06, 358459: 8.4e-06, 352699: 1

In [26]:
# print(all_keys[0])
for x in all_keys:
    if x not in final_ans.keys():
        final_ans[x] = 0
# print(372119 in final_ans.keys())
# print(final_ans[372119])

In [27]:
submit = pd.DataFrame(
    data={
        'alert_key': list(final_ans.keys()), 
        'probability': list(final_ans.values())
    }
)
submit.sort_values(by='probability', inplace=True)

In [28]:
print(submit)
submit.to_csv('submission.csv', index=None)

      alert_key  probability
1924     375178     0.000000
2558     374777     0.000000
2559     372891     0.000000
2560     372767     0.000000
2561     372329     0.000000
...         ...          ...
706      364601     0.001146
570      361158     0.001220
393      357567     0.001342
670      363315     0.001724
388      357485     0.005243

[3850 rows x 2 columns]
